In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A3_016, input_features_A3_016 = pre_models.Input_A3_016('weights/')
    
    outputs = ['Input_A3_016']

    preprocessings = ['min_max_normalization']

    models = [model_A3_016]

    input_features = [input_features_A3_016]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=8
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A3_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A3_019', 'Input_A2_019', 'Input_A6_016', 'Input_A1_016',
       'Input_A4_018', 'Input_A4_016', 'Input_A5_016', 'Input_A6_017'],
      dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.009291638928026555
[0.00917185120204778, 0.007859949837441165, 0.01000706470197451, 0.010140386307548635, 0.0079845428435788, 0.007658897972012282, 0.00922420102037196, 0.010109775800507512, 0.010503094384896024, 0.010256625209886877]


test_r2_ave: 0.527157932357246
[0.5674849782466548, 0.6471582038152566, 0.4677685437111081, 0.45324836330785245, 0.6281209591177148, 0.6697325483778992, 0.5221822411722052, 0.41202244413653166, 0.4223386765465226, 0.4815223651407139]


## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,8)

train_rmse: 0.008505921572252061
test_rmse: 0.009249846146396823
test_r2: 0.5436609914464399


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,9e-05)

test_rmse_ave: 0.009115614264934825
[0.009405237342328442, 0.007875143159502693, 0.009800286932834593, 0.009865856921933872, 0.007521914929789588, 0.007974452358010926, 0.009107450808243583, 0.009888716055640541, 0.01010274105743251, 0.009614343083631483]


test_r2_ave: 0.5456515961610326
[0.5451934467112469, 0.6457927955311251, 0.489536486155844, 0.48245190163035556, 0.6699661954808496, 0.6419572016995084, 0.5342011225180354, 0.43745468391050724, 0.4655375502695903, 0.5444245777032639]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.025],0.01)

test_rmse_ave: 0.009107383557525316
[0.009369592396116611, 0.007933388090389815, 0.009718423860535301, 0.009875318188896661, 0.007563815235390695, 0.007974343221045635, 0.009073351001855794, 0.009847562073597053, 0.010080229088624803, 0.009637812418800787]


test_r2_ave: 0.5466826244967145
[0.548634260466682, 0.6405339534212691, 0.4980288050150513, 0.48145877783177415, 0.6662790937798011, 0.6419670018551317, 0.5376826490963671, 0.4421272431910037, 0.4679167851085906, 0.5421976752014745]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,8,model)

train_rmse: 0.008763569204631106
test_rmse: 0.00955710219207867
test_r2: 0.5131310213629848


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 6, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 1.5)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.009594181753428889
[0.009576742820043815, 0.008234342727512917, 0.010343216552456923, 0.01008971077463508, 0.008743945685291465, 0.008308080533092684, 0.00906382909449514, 0.010520708709442201, 0.010856770512427484, 0.01020447012489119]


test_r2_ave: 0.4968166163077624
[0.5284553256161528, 0.6127438230159745, 0.4314111246846385, 0.4586993783798716, 0.5540188597801703, 0.6113715392511025, 0.5386524855108219, 0.3632518496415299, 0.38277988623707604, 0.486781890960287]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 9,
            min_samples_split = 7,
            max_leaf_nodes = 13,
            min_samples_leaf =3,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.009281953638784288
[0.00919788220044316, 0.0076671390524333015, 0.009223367614186576, 0.010113928440014321, 0.008533146233002335, 0.00826455322547545, 0.009207040273129932, 0.010028571290706478, 0.010237058213106089, 0.010346849845345226]


test_r2_ave: 0.5292924845334115
[0.5650264178946611, 0.6642568544048165, 0.547867047547295, 0.4560977637261481, 0.575263124918487, 0.6154330400667123, 0.5239584562350499, 0.4214301053719479, 0.4512315936565915, 0.47236044151240597]


In [11]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-05)
elastic_mod=ElasticNet(alpha=[0.025], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 6, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.5,
              reg_alpha =  0,
              reg_lambda = 1.5)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 9,
            min_samples_split = 7,
            max_leaf_nodes = 13,
            min_samples_leaf =3,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.009064258363411994
[0.00935509522883439, 0.007727383552072952, 0.0096216240294308, 0.00978491083154626, 0.0076419255598362676, 0.007981279750199998, 0.00894352707428756, 0.009843502091243251, 0.010047330646860253, 0.009696004869808207]


test_r2_ave: 0.5510014954220127
[0.5500299374818509, 0.6589599260002454, 0.507978717664354, 0.4909096837268001, 0.6593509384541075, 0.6413438567490715, 0.5508179205564308, 0.44258715126377834, 0.47138419518340013, 0.5366526271400879]


In [12]:

stack_mod = StackingRegressor(regressors=[lasso_mod,elastic_mod,lr, random_mod ], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.009163946587901189
[0.009162840429715532, 0.00754423895481944, 0.009271071912948575, 0.010000168602739841, 0.008100946078283685, 0.008148243439358886, 0.009034847558137276, 0.009997669544893882, 0.010316555544392174, 0.010062883813722598]


test_r2_ave: 0.5408279588957617
[0.5683343989101675, 0.674934149903187, 0.5431779872849749, 0.4682644019564126, 0.6171989909624112, 0.6261811488515499, 0.5415980802446482, 0.42499018859124416, 0.44267542187767495, 0.5009248203753472]


In [13]:
ave(skew_data[title],y,0.2,stack_mod,0.6,vote_mod,0.2,elastic_mod)

test_rmse_ave: 0.00905264716983455
[0.009285161844297401, 0.007699209475153332, 0.009547096850337923, 0.009803124085258064, 0.00766568724338029, 0.00797039885863066, 0.008933407388431472, 0.009840930250428202, 0.01007457514124538, 0.009706880561182785]


test_r2_ave: 0.552114728131019
[0.5567322340275322, 0.6614422598754238, 0.5155713949294537, 0.4890127179392084, 0.6572292272625186, 0.6423211031521858, 0.5518338530434843, 0.4428783870098505, 0.46851350305665473, 0.535612601013878]


In [14]:
name='Input_A3_016'
test_pred=[]
seed=[1,4,5,6,0]
stack_w=0.2
vote_w=0.6
model_w=0.2
model=elastic_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)



final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))